In [ ]:
# Only for google Colab
# !pip install cebra
# from google.colab import drive
# drive.mount('/content/drive/')
# !unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip


In [ ]:
# !unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_behavior.zip

In [ ]:
!mkdir 2020_11_9_MV1_run_behavior


In [ ]:
!mv *.tif 2020_11_9_MV1_run_behavior/

In [ ]:
!conda init bash
!conda activate cebra-env

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from cebra import CEBRA
from PIL import Image
import cv2
import torch


In [ ]:
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(DEVICE)

In [ ]:
from logging import exception
def process_brain(path):
  try:
    brain_seq = cv2.imreadmulti(path)[1]
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))


In [ ]:
# iterate through all brain data and process it
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os
neural_data = []
name_data = []
behav_data = []

filepath = "2020_11_9_MV1_run_brain"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         neural_data.append(process_brain(filepath + '/' + filename))
         name_data.append(filename)
         split_name = filename.split('_')
         session_data = []
         session_data.append(int(split_name[2].split('.')[0]))
         session_data.append(int(split_name[1]))
         behav_data.append(session_data)
         continue
     else:
         continue

In [ ]:
# using https://github.com/facebookresearch/dinov2
# extract feature labels from images
import torch


In [ ]:
# from https://huggingface.co/facebook/dino-vits8
# testing transformer
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('facebook/dino-vits8')
model = ViTModel.from_pretrained('facebook/dino-vits8')
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)
inputs = processor(images=image, return_tensors="pt").to(device)
outputs = model(**inputs)

In [ ]:
np.size(outputs.pooler_output.cpu().detach().numpy())

In [ ]:
def get_features(model, image):
  return model(**processor(images=image, return_tensors="pt").to(device)).pooler_output.cpu().detach().numpy()

In [ ]:
from PIL import Image
import numpy as np

def np_to_PIL(numpy_image):
    return Image.fromarray(np.uint8(numpy_image)).convert('RGB')

In [ ]:
def process_behav(path):
  beh_seq = cv2.imreadmulti(path)[1]
  beh_seq = np.array(beh_seq)
  feature_seq = []
  for beh_feat in beh_seq:
    feature_seq.append(get_features(model, np_to_PIL(beh_feat)))
  return np.array(feature_seq)


In [ ]:
process_behav("2020_11_9_MV1_run_behavior/move_0001_0001.tif")

In [ ]:
behav_feature = []
name_data_behav = []

filepath = "2020_11_9_MV1_run_behavior"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         behav_feature.append(process_behav(filepath + '/' + filename))
         name_data_behav.append(filename)
         continue
     else:
         continue

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('feature_labels', 'wb')

# dump information to that file
pickle.dump(behav_feature, file)

# close the file
file.close()

In [ ]:
import pickle
with open('feature_labels', 'rb') as f:
    behav_feature = pickle.load(f)


In [ ]:
out_dim = 8
max_iterations = 10

In [ ]:
multi_cebra_model = CEBRA(batch_size=128,
                                output_dimension=out_dim,
                                max_iterations=20000,
                                verbose=True,
                                temperature_mode="auto",
                                device = "cuda_if_available",
                                max_adapt_iterations=10)

In [ ]:
behav_data2 = []
for data in behav_feature:
    behav_data2.append(np.squeeze(data))
behav_feature = behav_data2


In [ ]:
#set aside 20% of data for validation
neural_data_train = neural_data[:(int)(282 * 0.8)]
behav_feature_train = behav_feature[:(int)(282 * 0.8)]
print(len(neural_data_train))

In [ ]:
multi_cebra_model.fit(neural_data, behav_feature)
multi_cebra_model.save("cebra_multi_model2.pt")

In [ ]:
print(np.shape(mock_behav[1]))
np.shape(neural_data[1])

In [ ]:

import cebra
fig = plt.figure()

ax1 = plt.subplot(141, projection='3d')
ax2 = plt.subplot(142, projection='3d')
index = 20
for run, data in enumerate(zip(neural_data[:index], name_data[:index])):
  is_move = True
  if data[1].split('_')[0] == 'move':
    is_move = True
  else:
    is_move = False
  embedding = multi_cebra_model.transform(data[0], session_id=run)
  if is_move:
    ax1 = cebra.plot_embedding(embedding, ax=ax1, title='moving')
  else:
    ax2 = cebra.plot_embedding(embedding, ax=ax2, title = 'no move')

plt.show()


In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)

ax = cebra.plot_loss(multi_cebra_model, color='deepskyblue', label='loss', ax=ax)
plt.show()

In [ ]:
from IPython.display import Video

In [ ]:
import sklearn.manifold._t_sne as TSNE
index = 1
dino_tsne = TSNE.TSNE(n_components = 2)
dino_tsne_viz = dino_tsne.fit_transform(behav_feature[index])
fig = plt.figure(figsize = (5,5))
plt.scatter(dino_tsne_viz[:,0], dino_tsne_viz[:,1], cmap = 'magma', c = np.arange(len(behav_feature[index])))
plt.title(name_data[index])
plt.axis('off')
frameSize = (320,320)
out = cv2.VideoWriter('output_video.mp4',cv2.VideoWriter_fourcc(*'XVID'), 30, frameSize)
for image in image_data[index]:
    out.write(image)
Video("output_video.mp4")

In [ ]:
cv2.imshow("image",image_data[0][2])
k = cv2.waitKey(0)